In [1]:
import fridge
import fridge.utilities.h5Interface as h5i
import fridge.utilities.reactorInterface as RI
import os

from scipy.interpolate import interp1d as interp
from matplotlib import pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

import osbrain
from osbrain import run_nameserver
from osbrain import run_agent
from osbrain import logging
import numpy as np
import blackboard
import ka
import ka_rp as karp
import ka_br as kabr
import bb_basic
import time

# SFR Optimization

This example seeks to optimize a SFR design utilizing the blackboard framework.
We start first by taking a look at the SFR design data using the `h5` add `reactor` interfaces built into `FRDIGe` [cite].
For this we'll examine the objectives that will drive the optimization process: cycle length, plutonium mass, reactivity swing, and core average burnup.
Upon examination, we note that we have removed a significant portion of the design space by ensuring that the reactor design has a cycle length greater than 0 days.

In [2]:
h5_interface = h5i.h5Interface()
h5_interface.read_h5('SFR_DB.h5')

In [3]:
cycle_length_list = []
rx_swing_list = []
height = []
smear = []
pu_content = []
bu = []
pu_mass = []
for rx, core in h5_interface.h5file.items():
    try:
        rx_ = RI.reactorInterface(core)
        cycle_length = rx_.extrapolate_value('keff', 'time', 1.0)
        rx_swing = rx_.get_reactivity_swing(1.0, cycle_length)
        rx_swing_per_day = rx_swing/cycle_length
        avg_bu = rx_.get_assembly_avg(cycle_length, 'burnup')
        avg_bu_per_day = avg_bu/cycle_length
        pu = rx_.get_assembly_pu_mass() * 78 / 4
        if cycle_length > 0:
            height.append(rx_.rx['independent variables']['height'][0])
            smear.append(rx_.rx['independent variables']['smear'][0])
            pu_content.append(rx_.rx['independent variables']['pu_content'][0])
            cycle_length_list.append(cycle_length)
            rx_swing_list.append(rx_swing_per_day)
            bu.append(avg_bu_per_day)
            pu_mass.append(pu)
    except (KeyError, TypeError):
        print('didnt get {}'.format(core))

50.0 50.0 0.253144227664784 15.692999777 0.555555 87.75845399234733
50.0 50.0 0.253144227664784 15.77105 0.75 119.06327017712319
50.0 50.0 0.253144227664784 15.871400000000001 1.0 159.7611476969251
50.0 50.0 0.253144227664784 15.47 0 0.0
50.0 50.0 0.253144227664784 15.570350000000001 0.25 39.18269632866064
60.0 50.0 0.253144227664784 15.692999777 0.555555 105.31014479081679
60.0 50.0 0.253144227664784 15.77105 0.75 142.87592421254783
60.0 50.0 0.253144227664784 15.871400000000001 1.0 191.71337723631012
60.0 50.0 0.253144227664784 15.47 0 0.0
60.0 50.0 0.253144227664784 15.570350000000001 0.25 47.01923559439276
70.0 50.0 0.253144227664784 15.692999777 0.555555 122.86183558928626
70.0 50.0 0.253144227664784 15.77105 0.75 166.68857824797243
70.0 50.0 0.253144227664784 15.871400000000001 1.0 223.66560677569512
70.0 50.0 0.253144227664784 15.47 0 0.0
70.0 50.0 0.253144227664784 15.570350000000001 0.25 54.85577486012489
80.0 50.0 0.253144227664784 15.692999777 0.555555 140.41352638775572
80.

In [7]:
fig = px.scatter_3d(x=height, y=smear, z=cycle_length_list, color=pu_content, labels={'x':'Height (cm)', 'y': 'Smear', 'z':'Cycle Length (days)','color':'Pu Fraction'})
fig.show()
fig2 = px.scatter_3d(x=height, y=smear, z=rx_swing_list, color=pu_content, labels={'x':'Height (cm)', 'y': 'Smear', 'z':'Rx Swing (pcm)','color':'Pu Fraction'})
fig2.show()
fig3 = px.scatter_3d(x=height, y=smear, z=bu, color=pu_content, labels={'x':'Height (cm)', 'y': 'Smear', 'z':'Burnup (GWd/MTU)','color':'Pu Fraction'})
fig3.show()
fig4 = px.scatter_3d(x=height, y=smear, z=pu_mass, color=pu_content, labels={'x':'Height (cm)', 'y': 'Smear', 'z':'Pu Mass per Cycle (kg)','color':'Pu Fraction'})
fig4.show()

# Starting the Blackboard System

In [2]:
ns = run_nameserver()
bb = run_agent(name='bb', base=bb_basic.BbSfrOpt)

bb.connect_agent(karp.KaRpExplore, 'ka_rp_explore')
bb.connect_agent(kabr.KaBr_lvl2, 'ka_br_lvl2')
bb.connect_agent(kabr.KaBr_lvl3, 'ka_br_lvl3')

Broadcast server running on 0.0.0.0:9091
NS running on 127.0.0.1:17716 (127.0.0.1)
URI = PYRO:Pyro.NameServer@127.0.0.1:17716
INFO [2020-04-28 17:05:14.423265] (bb): Connected agent ka_rp_explore of agent type <class 'ka_rp.KaRpExplore'>
INFO [2020-04-28 17:05:14.475584] (bb): Connected agent ka_br_lvl2 of agent type <class 'ka_br.KaBr_lvl2'>
INFO [2020-04-28 17:05:14.528661] (bb): Connected agent ka_br_lvl3 of agent type <class 'ka_br.KaBr_lvl3'>
INFO [2020-04-28 17:05:17.550403] (bb): Selecting agent ka_rp_explore (TV: 1.0) to execute (TE: 1)
INFO [2020-04-28 17:05:17.553664] (ka_rp_explore): Core Design & Objectives: [('height', 72.82), ('smear', 58.07), ('pu_content', 0.43), ('cycle length', 213.02257367244954), ('reactivity swing', 5076.728440083846), ('burnup', 33.58043361258371), ('pu mass', 598.1282719948757)]
INFO [2020-04-28 17:05:20.561109] (bb): Agent ka_rp_explore given permission to write
INFO [2020-04-28 17:05:20.565751] (bb): No KA to execute, waiting to sends trigger a

In [3]:
while not bb.get_attr('_complete'):
    bb.publish_trigger()
    bb.controller()
    bb.send_executor()
    bb.wait_for_ka()
    print('Finished trigger event: {}'.format(bb.get_attr('_trigger_event')))    

Finished trigger event: 1
Finished trigger event: 2
Finished trigger event: 3
Finished trigger event: 4
Finished trigger event: 5
Finished trigger event: 6
Finished trigger event: 7
Finished trigger event: 8
Finished trigger event: 9
Finished trigger event: 10
Finished trigger event: 11
Finished trigger event: 12
Finished trigger event: 13
Finished trigger event: 14
Finished trigger event: 15
Finished trigger event: 16
Finished trigger event: 17
Finished trigger event: 18
Finished trigger event: 19
Finished trigger event: 20
Finished trigger event: 21
Finished trigger event: 22
Finished trigger event: 23
Finished trigger event: 24
Finished trigger event: 25
Finished trigger event: 26
Finished trigger event: 27
Finished trigger event: 28
Finished trigger event: 29
Finished trigger event: 30
Finished trigger event: 31
Finished trigger event: 32
Finished trigger event: 33
Finished trigger event: 34
Finished trigger event: 35
Finished trigger event: 36
Finished trigger event: 37
Finished t

KeyboardInterrupt: 